In [15]:
import scipy as sp
import numpy as np
import pandas as pd
import timeit
import re
import json
import pickle
import fastparquet
import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')
seed = 3
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.iolib.summary2 as summary2
import logging

Based on https://sfengc7.stern.nyu.edu:8888/notebooks/patentdiffusion/201808Results/Reg0919/1-CleanRegData-0919.ipynb

In [18]:
# Create new datasets and save
pathdir = "DataStore/2018-07-P2/Reg0726/"
mod_keys = ["naics_name", "primclass"]
# mod_keys = ["primclass"]

for k in mod_keys:
    print(k)
    if k == "naics_name":
        req_cols = ['tp', 'op',
            'sim_docvecs', 'sim_ldavecs', 'tp_gyear', 'tp_naics_name',
           'tp_primclass', 'tp_inv_msa', 'op_naics_name', 'op_primclass',
           'op_inv_msa', 'inv_msa_match', 'primclass_match',
           'norm_sim_ldavecs', 'norm_sim_docvecs', 'sim_mean_ldavecs_pc_msa',
           'sim_mean_docvecs_pc_msa', 'sim_mean_docvecs_pc', 'sim_mean_ldavecs_pc',
           'norm_sim_mean_ldavecs_pc_msa', 'norm_sim_mean_docvecs_pc_msa', 'norm_sim_mean_docvecs_pc', 
            'norm_sim_mean_ldavecs_pc', 'year_group']
    else:
        req_cols = ['tp', 'op',
            'sim_docvecs', 'sim_ldavecs', 'tp_gyear', 'tp_naics_name',
           'tp_primclass', 'tp_inv_msa', 'op_naics_name', 'op_primclass',
           'op_inv_msa', 'inv_msa_match', 'primclass_match',
           'norm_sim_ldavecs', 'norm_sim_docvecs', 'year_group', 'sim_mean_ldavecs_pc_msa',
           'sim_mean_docvecs_pc_msa', 'norm_sim_mean_ldavecs_pc_msa',
           'norm_sim_mean_docvecs_pc_msa']
    # Load
    rs = fastparquet.ParquetFile(pathdir+"reg_{0}_sim_tr_0726.parq".format(k)).to_pandas(req_cols)
    #rename columns
    rs = rs.rename(columns={'sim_mean_ldavecs_pc_msa': 'mean_sim_ldavecs_pc_msa_v',
                            'sim_mean_docvecs_pc_msa': 'mean_sim_docvecs_pc_msa_v', 
                            'sim_mean_docvecs_pc': 'mean_sim_docvecs_pc_v', 
                            'sim_mean_ldavecs_pc': 'mean_sim_ldavecs_pc_v',
                            'norm_sim_mean_ldavecs_pc_msa': 'norm_mean_sim_ldavecs_pc_msa_v',
                            'norm_sim_mean_docvecs_pc_msa': 'norm_mean_sim_docvecs_pc_msa_v',
                            'norm_sim_mean_docvecs_pc': 'norm_mean_sim_docvecs_pc_v', 
                            'norm_sim_mean_ldavecs_pc': 'norm_mean_sim_ldavecs_pc_v'})
    
    rs2 = fastparquet.ParquetFile(pathdir+"{0}_common_inv_0726.parq".format(k))\
    .to_pandas(["tp", "op", "common_est_inv", "common_pat_inv", "lawyer_match",\
                "num_common_cited", "norm_num_common_cited", "tp_pct_common_cited",
                "norm_tp_pct_common_cited", "common_cited_match"])
    rs = rs.merge(rs2, how="left", on = ["tp", "op"])
    del(rs2)
    
#     if k == "naics_name":
#         # Get tp-pc similarities
#         rs3 = pd.read_pickle(pathdir+"naics_name_tp_pc_msa_0824.pkl")[["tp", "sim_mean_ldavecs_tp_pc_msa", "sim_mean_docvecs_tp_pc_msa"]]
#         rs4 = pd.read_pickle(pathdir+"naics_name_tp_pc_0824.pkl")[["tp", "sim_mean_ldavecs_tp_pc", "sim_mean_docvecs_tp_pc"]]
#         rs3 = rs3.merge(rs4, how="left", on = "tp").drop_duplicates()
#         del(rs4)

#         for c in ["sim_mean_ldavecs_tp_pc_msa", "sim_mean_docvecs_tp_pc_msa",
#                  "sim_mean_ldavecs_tp_pc", "sim_mean_docvecs_tp_pc_msa"]:
#             # Get normed values for all similarities
#             rs3["norm_{0}".format(c)] = np.nan
#             rs3.loc[rs3[c].notnull(), "norm_{0}".format(c)] = \
#             scaler.fit_transform(rs3.loc[rs3[c].notnull(), c].values.reshape(-1,1))

#         # Merge back with original
#         print(len(rs))
#         rs = rs.merge(rs3, how="left", on = "tp")
#         del(rs3)
#     print(rs.columns)
#     print(len(rs))
    
    # Add Mean Sim: Mean of past similarity
    ps = fastparquet.ParquetFile("DataStore/2018-10/pc_sim_pairs_0929.parq").to_pandas()
    
#     # Normalize similarities: remember to do ONLY FOR UNIQUE VALUES FIRST
#     for c in ["mean_sim_ldavecs_pc", "mean_sim_docvecs_pc", "mean_sim_ldavecs_pc_msa", "mean_sim_docvecs_pc_msa"]:
#             if (c == "mean_sim_ldavecs_pc") or (c == "mean_sim_docvecs_pc"):
#                 # Drop duplicate observations on the index
#                 p2 = ps[["tp_primclass", "op_primclass", "tp_gyear", c]]\
#                 .dropna(how="any").drop_duplicates(["tp_primclass", "op_primclass", "tp_gyear"])
                
#             else:
#                 p2 = ps[["tp_primclass", "tp_inv_msa", "tp_gyear",
#                         "op_primclass", "op_inv_msa", c]]\
#                 .dropna(how="any").drop_duplicates(["tp_primclass", "tp_inv_msa", "tp_gyear",\
#                                               "op_primclass", "op_inv_msa"])
#                 print(len(ps))
                
#             print(len(ps), len(p2))
#             # Fit
#             scaler = StandardScaler().fit(p2[c].values.reshape(-1,1))

#             #Transform
#             ps.loc[ps[c].notnull(), "norm_{0}".format(c)] = \
#             scaler.transform(ps.loc[ps[c].notnull(), c].values.reshape(-1,1))
#             del(p2)
    
    rs = rs.merge(ps, how="left", on=["tp_primclass", "tp_inv_msa", "tp_gyear",
                                              "op_primclass", "op_inv_msa"])
    print(len(rs))
    
    # Indicator for being in same MSA and having mean_sim_docvecs_pc above 1
    rs["pc_msa_greater_0"] = (rs["norm_mean_sim_docvecs_pc_msa"] >= 0)
    rs["pc_msa_greater_0"] = rs["pc_msa_greater_0"].astype(int)
    rs["pc_msa_less_0"] = 1-rs["pc_msa_greater_0"]
    
    # Save
    fastparquet.write("DataStore/2018-10/Reg0930/{0}_all_1004.parq".format(k), rs, compression="GZIP")
    
    del(rs)

naics_name
1498184
primclass
1369833


In [19]:

fe = {
    
     "No FE": "{0} ~ C(inv_msa_match) + C(tp_gyear)",
     "PC FE": "{0} ~ C(inv_msa_match) + C(tp_gyear) + C(tp_primclass)",
     "Inv FE": "{0} ~ C(inv_msa_match) + C(tp_gyear) + C(tp_primclass) + C(common_est_inv) + C(common_pat_inv)",
     "Lawyer FE": "{0} ~ C(inv_msa_match) + C(tp_gyear) + C(tp_primclass) + C(common_est_inv) + C(common_pat_inv) + \
     C(lawyer_match)",
     "All FE": "{0} ~ C(inv_msa_match) + C(tp_gyear) + C(tp_primclass) + C(common_est_inv) + C(common_pat_inv) + \
     C(lawyer_match) + C(primclass_match)",
}

comp_fe = {
     
     "Inv M": "{0} ~ C(common_pat_inv) + C(tp_gyear)",
     "Inv M-PC FE": "{0} ~ C(common_pat_inv) + C(tp_gyear) + C(tp_primclass)",
     "Lawyer M": "{0} ~ C(lawyer_match) + C(tp_gyear)",
     "Lawyer M-PC FE": "{0} ~ C(lawyer_match) + C(tp_gyear) + C(tp_primclass)",
     "CC M": "{0} ~ C(common_cited_match) + C(tp_gyear)",
     "CC M-PC FE": "{0} ~ C(common_cited_match) + C(tp_gyear) + C(tp_primclass)",
     "PC M": "{0} ~ C(primclass_match) + C(tp_gyear)",
     "PC M-PC FE": "{0} ~ C(primclass_match) + C(tp_gyear) + C(tp_primclass)",
}

techsim = {
     # All sim
    
    "Sim PC": "{0} ~ C(inv_msa_match) + C(tp_gyear) + mean_{0}_pc",
    
    "Int PC": "{0} ~ C(inv_msa_match)*mean_{0}_pc + C(tp_gyear)",

    "Sim PC MSA": "{0} ~ C(inv_msa_match) + C(tp_gyear) + mean_{0}_pc_msa",

    "Int PC MSA": "{0} ~ C(inv_msa_match)*mean_{0}_pc_msa + C(tp_gyear)",
    
    # PC MSA Cubic
    "Sim PC MSA-Cb": "{0} ~ C(inv_msa_match) + C(tp_gyear) + mean_{0}_pc_msa + np.power(mean_{0}_pc_msa, 3)",
    
#     "Sim PC MSA-Break Cb": "{0} ~ C(inv_msa_match) + C(tp_gyear) + mean_{0}_pc_msa + np.power(mean_{0}_pc_msa, 3)\
#     + MSA_Match_Break:np.power(mean_{0}_pc_msa, 3)",
    
     "Int PC MSA-Add Cb": "{0} ~ C(inv_msa_match)*mean_{0}_pc_msa + np.power(mean_{0}_pc_msa,3) + C(tp_gyear)",
     
#      "Int PC MSA-Break Cb": "{0} ~ C(inv_msa_match)*mean_{0}_pc_msa + np.power(mean_{0}_pc_msa,3) +\
#      + MSA_Match_Break:np.power(mean_{0}_pc_msa, 3) + C(tp_gyear)",
    
     "Int PC MSA-Int Cb": "{0} ~ C(inv_msa_match)*mean_{0}_pc_msa + C(inv_msa_match)*np.power(mean_{0}_pc_msa,3)\
     + C(tp_gyear)",
    
    "Break Int PC MSA": "{0} ~ C(inv_msa_match) + pc_msa_greater_0:mean_{0}_pc_msa + \
    pc_msa_less_0:mean_{0}_pc_msa + C(inv_msa_match):pc_msa_greater_0:mean_{0}_pc_msa + \
    C(inv_msa_match):pc_msa_less_0:mean_{0}_pc_msa + C(tp_gyear)"
}
    
pc_fe_techsim = {"PC FE-"+k : v+" + C(tp_primclass)" for k,v in techsim.items()}
all_fe_techsim = {"All FE-"+k : v+" + C(tp_primclass) + C(common_est_inv) + \
C(common_pat_inv) + C(lawyer_match) + C(primclass_match)" for k,v in techsim.items()}   
inv_fe_techsim = {"Inv FE-"+k : v+" + C(tp_primclass) + C(common_est_inv) + \
C(common_pat_inv) + C(lawyer_match)" for k,v in techsim.items()}  
inv_fe_pc_techsim = {"Inv FE-"+k : v+" + C(common_est_inv) + \
C(common_pat_inv) + C(lawyer_match) + C(primclass_match)" for k,v in techsim.items()}  

# Models
s1 = list(fe.values())+list(comp_fe.values())+list(all_fe_techsim.values())+list(inv_fe_techsim.values())
s2 = [i.replace("{0} ", "norm_{0} ").replace("mean_", "norm_mean_") for i in s1]
sim_regs = s1+s2

n_eqns = len(s1)

# Model names
m1 = list(fe.keys())+list(comp_fe.keys())+list(all_fe_techsim.keys())+list(inv_fe_techsim.keys())
# Add Norm
m2 = ["N "+i for i in m1]
mn = m1+m2

In [20]:
# Create regression dictionary
regs = {}
pathdir = "DataStore/2018-10/Reg0930/"
reg_f = "reg_model_1002.pkl"

regs["model_names"] = pd.Series(mn)
regs["n_eqns"] = n_eqns
regs["ldavecs"] = pd.Series([i.format("sim_ldavecs") for i in sim_regs])
regs["docvecs"] = pd.Series([i.format("sim_docvecs") for i in sim_regs])

# Include mean_sim_docvecs
for c in ["num_common_cited", "tp_pct_common_cited"]:
    # Just replace KS measure
    l = [i.replace("sim_docvecs ", "{0} ".format(c)) for i in regs["docvecs"]]
    regs[c] = pd.Series(l)
pickle.dump(regs, open(pathdir+reg_f, "wb"))